In [2]:
from tensorflow.keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

In [3]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [5]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

base_dir = '/root/data/cats_and_dogs_small'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            # Note that since generators yield data indefinitely in a loop,
            # we must `break` after every image has been seen once.
            break
    return features, labels

train_features, train_labels = extract_features(train_dir, 2000)
validation_features, validation_labels = extract_features(validation_dir, 1000)
test_features, test_labels = extract_features(test_dir, 1000)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [6]:
train_features = np.reshape(train_features, (2000, 4 * 4 * 512))
validation_features = np.reshape(validation_features, (1000, 4 * 4 * 512))
test_features = np.reshape(test_features, (1000, 4 * 4 * 512))

In [7]:
from tensorflow.keras import models,layers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [8]:
conv_base.trainable = False

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=50,
      verbose=2)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/30
100/100 - 13s - loss: 0.5931 - acc: 0.6880 - val_loss: 0.4438 - val_acc: 0.8130
Epoch 2/30
100/100 - 12s - loss: 0.4831 - acc: 0.7835 - val_loss: 0.3652 - val_acc: 0.8560
Epoch 3/30
100/100 - 12s - loss: 0.4278 - acc: 0.8210 - val_loss: 0.3342 - val_acc: 0.8540
Epoch 4/30
100/100 - 12s - loss: 0.3911 - acc: 0.8315 - val_loss: 0.3060 - val_acc: 0.8770
Epoch 5/30
100/100 - 12s - loss: 0.3846 - acc: 0.8340 - val_loss: 0.3163 - val_acc: 0.8590
Epoch 6/30
100/100 - 12s - loss: 0.3683 - acc: 0.8380 - val_loss: 0.2920 - val_acc: 0.8710
Epoch 7/30
100/100 - 12s - loss: 0.3504 - acc: 0.8430 - val_loss: 0.2836 - val_acc: 0.8780
Epoch 8/30
100/100 - 12s - loss: 0.3488 - acc: 0.8445 - val_loss: 0.2780 - val_acc: 0.8840
Epoch 9/30
100/100 - 12s - loss: 0.3360 - acc: 0.8600 - val_loss: 0.2899 - val_acc: 0.8700
Epoch 10/30
100/100 - 12s - loss: 0.3359 - acc: 0.8570 - val_loss: 0.2661 - val_acc: 0.8980
Epoch

In [11]:
model.save('/root/models/cats_and_dogs_small_3.h5')

In [12]:
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [13]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])

history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=100,
      validation_data=validation_generator,
      validation_steps=50)

Epoch 1/100
100/100 [==============================] - 12s 124ms/step - loss: 0.2946 - acc: 0.8695 - val_loss: 0.2289 - val_acc: 0.9100
Epoch 2/100
100/100 [==============================] - 12s 121ms/step - loss: 0.2502 - acc: 0.8975 - val_loss: 0.2163 - val_acc: 0.9120
Epoch 3/100
100/100 [==============================] - 12s 122ms/step - loss: 0.2378 - acc: 0.8970 - val_loss: 0.2236 - val_acc: 0.9130
Epoch 4/100
100/100 [==============================] - 12s 122ms/step - loss: 0.2194 - acc: 0.9085 - val_loss: 0.2060 - val_acc: 0.9150
Epoch 5/100
100/100 [==============================] - 12s 121ms/step - loss: 0.2105 - acc: 0.9140 - val_loss: 0.2086 - val_acc: 0.9150
Epoch 6/100
100/100 [==============================] - 12s 122ms/step - loss: 0.1987 - acc: 0.9160 - val_loss: 0.2289 - val_acc: 0.9130
Epoch 7/100
100/100 [==============================] - 12s 122ms/step - loss: 0.1754 - acc: 0.9280 - val_loss: 0.1860 - val_acc: 0.9260
Epoch 8/100
100/100 [===========================

In [14]:
model.save('/root/models/cats_and_dogs_small_4.h5')

In [15]:
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)
print('test acc:', test_acc)

Found 1000 images belonging to 2 classes.
test acc: 0.932
